# 1. Setup

### Links

Jirka's links:
* [Spine🦴Fracture: EDA🔎 & loading DICOM & 3D browse](https://www.kaggle.com/code/jirkaborovec/spine-fracture-eda-loading-dicom-3d-browse)
* [Spine🦴Fracture: convert🤖 DICOM imgs -> 3D volume](https://www.kaggle.com/code/jirkaborovec/spine-fracture-convert-dicom-imgs-3d-volume)
* [Spine🦴Fracture: convert🤖 DICOM -> equalized PNG](https://www.kaggle.com/code/jirkaborovec/spine-fracture-convert-dicom-equalized-png)
* [SpineFrac🦴Classif: e2e ~ Lightning⚡MONAI⚕️3D](https://www.kaggle.com/code/jirkaborovec/spinefrac-classif-e2e-lightning-monai-3d/notebook)
* [Cervical Spine Fracture Detection: 3D volumes](https://www.kaggle.com/datasets/jirkaborovec/cervical-spine-fracture-detection-npz-3d-volumes)
* [Cervical Spine Fracture Detection: equalized PNG](https://www.kaggle.com/datasets/jirkaborovec/cervical-spine-fracture-detection-equalized-png)

Sam's links:

* [🦴 RSNA Fracture Detection - in-depth EDA](https://www.kaggle.com/code/samuelcortinhas/rsna-fracture-detection-in-depth-eda)
* [Extracting Vertebrae C1, ..., C7](https://www.kaggle.com/code/samuelcortinhas/extracting-vertebrae-c1-c7)
* [RSNA - CT gifs](https://www.kaggle.com/code/samuelcortinhas/rsna-ct-gifs)
* [RSNA 2022 Spine Fracture Detection - Metadata](https://www.kaggle.com/datasets/samuelcortinhas/rsna-2022-spine-fracture-detection-metadata)
* [RSNA - 3D train tensors [first half]](https://www.kaggle.com/datasets/samuelcortinhas/rsna-3d-train-tensors-first-half)
* [RSNA - 3D train tensors [second half]](https://www.kaggle.com/datasets/samuelcortinhas/rsna-3d-train-tensors-second-half)
* [RNSA - 3D model [Train] [PyTorch]](https://www.kaggle.com/code/samuelcortinhas/rnsa-3d-model-train-pytorch)
* [RSNA - Trained 3D model weights [PyTorch]](https://www.kaggle.com/datasets/samuelcortinhas/rsna-trained-3d-model-weights-pytorch)

### Libraries

In [ ]:
!pip install -qU ../input/for-pydicom/python_gdcm-3.0.14-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl ../input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl --find-links frozen_packages --no-index

In [ ]:
!pip install -q kaggle_vol3d_classify -f ../input/cervical-spine-fracture-detection-npz-3d-volumes/frozen_packages --no-index
# !pip install -qU "pytorch-lightning>1.5.0" --no-index
#!pip uninstall -y torchtext
#!pip list | grep -e lightning -e kaggle -e monai

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.6)
import cv2
import os
from os import listdir
import re
import gc
import random
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.auto import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
#warnings.filterwarnings("ignore", category=FutureWarning)
import zipfile
from scipy import ndimage
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from PIL import Image
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from kaggle_volclassif.utils import interpolate_volume
from skimage import exposure

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F

### Reproducibility

In [ ]:
# Set random seeds
def set_seed(seed=0):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
set_seed()

### Config

In [ ]:
# Hyperparameters
BATCH_SIZE = 1

# Config device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# 2. Data

### Load tables

In [ ]:
# Load metadata
train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
train_bbox = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
test_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/test.csv")
ss = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv")

# Print dataframe shapes
print('train shape:', train_df.shape)
print('train bbox shape:', train_bbox.shape)
print('test shape:', test_df.shape)
print('ss shape:', ss.shape)
print('')

# Show first few entries
train_df.head(3)

### Debug

In [ ]:
debug = False
if len(ss)==3:
    debug = True
    
    # Fix mismatch with test_images folder
    test_df = pd.DataFrame(columns = ['row_id','StudyInstanceUID','prediction_type'])
    for i in ['1.2.826.0.1.3680043.22327','1.2.826.0.1.3680043.25399','1.2.826.0.1.3680043.5876']:
        for j in ['C1','C2','C3','C4','C5','C6','C7','patient_overall']:
            test_df = test_df.append({'row_id':i+'_'+j,'StudyInstanceUID':i,'prediction_type':j},ignore_index=True)
    
    # Sample submission
    ss = pd.DataFrame(test_df['row_id'])
    ss['fractured'] = 0.5
    
    display(test_df.head(3))

### Load volumes

In [ ]:
# Convert dicom images to 3d tensor
def convert_volume(dir_path, out_dir = "test_volumes", size = (224, 224, 224)):
    ls_imgs = glob(os.path.join(dir_path, "*.dcm"))
    ls_imgs = sorted(ls_imgs, key=lambda p: int(os.path.splitext(os.path.basename(p))[0]))

    imgs = []
    for p_img in ls_imgs:
        dicom = pydicom.dcmread(p_img)
        img = apply_voi_lut(dicom.pixel_array, dicom)
        img = cv2.resize(img, size[:2], interpolation=cv2.INTER_LINEAR)
        imgs.append(img.tolist())
    vol = torch.tensor(imgs, dtype=torch.float32)

    vol = (vol - vol.min()) / float(vol.max() - vol.min())
    vol = interpolate_volume(vol, size).numpy()
    
    # https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_adapt_hist_eq_3d.html
    vol = exposure.equalize_adapthist(vol, kernel_size=np.array([64, 64, 64]), clip_limit=0.01)
    # vol = exposure.equalize_hist(vol)
    vol = np.clip(vol * 255, 0, 255).astype(np.uint8)
    
    path_pt = os.path.join(out_dir, f"{os.path.basename(dir_path)}.pt")
    torch.save(torch.tensor(vol), path_pt)

In [ ]:
# Make directory
os.mkdir('/kaggle/working/test_volumes')

# Get paths
ls_dirs = [p for p in glob(os.path.join("../input/rsna-2022-cervical-spine-fracture-detection", "test_images", "*")) if os.path.isdir(p)]
print(f"volumes: {len(ls_dirs)}")

# Convert volumes
_= Parallel(n_jobs=3)(delayed(convert_volume)(p_dir, out_dir='/kaggle/working/test_volumes') for p_dir in tqdm(ls_dirs))

### Torch dataset

In [ ]:
# Dataset for test set only
class RSNADataset(Dataset):
    # Initialise
    def __init__(self, subset='test', df_table=test_df):
        super().__init__()
        
        self.subset = subset
        self.df_table = df_table
        
        # Image paths
        self.volume_dir = '/kaggle/working/test_volumes/'
        
    # Get item in position given by index
    def __getitem__(self, index):
        
        # load 3d volume
        patient = self.df_table.loc[index,'StudyInstanceUID']
        path = os.path.join(self.volume_dir, f'{patient}.pt')
        vol = torch.load(path).to(torch.float32)
        
        return (vol.unsqueeze(0), patient)

    # Length of dataset
    def __len__(self):
        return len(self.df_table)

In [ ]:
# Test dataset
test_table = pd.DataFrame(pd.unique(test_df['StudyInstanceUID']),columns=['StudyInstanceUID'])
test_dataset = RSNADataset(subset='test', df_table = test_table)

### Torch dataloader

In [ ]:
# Dataloader
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 3. Model

conv output size = floor((W-F+2P)/S + 1)

In [ ]:
# 3D convolutional neural network
class Conv3DNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layers
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=7, stride=1, padding=0)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
        self.norm1 = nn.BatchNorm3d(num_features=16)
        self.conv2 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.norm2 = nn.BatchNorm3d(num_features=32)
        self.conv3 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)
        self.norm3 = nn.BatchNorm3d(num_features=64)
        self.avg = nn.AdaptiveAvgPool3d((7, 1, 1))
        self.flat = nn.Flatten()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(in_features=448, out_features=128)
        self.lin2 = nn.Linear(in_features=128, out_features=8)
        
    def forward(self, x):
        # Conv block 1
        out = self.conv1(x)
        out = self.relu(out)
        out = self.pool(out)
        out = self.norm1(out)
        
        # Conv block 2
        out = self.conv2(out)
        out = self.relu(out)
        out = self.pool(out)
        out = self.norm2(out)
        
        # Conv block 3
        out = self.conv3(out)
        out = self.relu(out)
        out = self.pool(out)
        out = self.norm3(out)
        
        # Average & flatten
        out = self.avg(out)
        out = self.flat(out)
        
        # Fully connected layer
        out = self.lin1(out)
        out = self.relu(out)
        
        # Output layer (no sigmoid needed)
        out = self.lin2(out)
        
        return out

model = Conv3DNet().to(device)

### Load model

In [ ]:
# Load checkpoint
PATH='../input/rsna-trained-3d-model-weights-pytorch/Conv3DNet.pt'
if torch.cuda.is_available():
    checkpoint = torch.load(PATH)
else:
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))

# Load states
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
val_loss = checkpoint['val_loss']

# Evaluation mode
model.eval()
model.to(device)

In [ ]:
# Print final loss and epoch
print('Final epoch:', epoch)
print('Final loss:', loss)
print('Final valid loss:', val_loss)

### Inference on test set

In [ ]:
test_df['fractured']=0.5
with torch.no_grad():
    # Loop over batches
    for i, (imgs, patient) in enumerate(test_loader):
        print(f'Iteration {i+1}/{len(test_loader)}')
        # Send to device
        imgs = imgs.to(device)
        
        # Make predictions
        preds = model(imgs)
        
        # Apply sigmoid
        sig = nn.Sigmoid()
        preds = sig(preds)
        preds = preds.to('cpu')
        
        # Save preds
        test_df.loc[test_df['StudyInstanceUID']==patient[0],'fractured'] = preds.numpy().squeeze()
        
print('Inference complete!')

### Submission

In [ ]:
submission = test_df[['row_id','fractured']]
submission.to_csv('submission.csv', index=False)
submission.head(3)